# setup

## import python modules

In [ ]:
import pandas as pd
import os

## define bucket location

In [ ]:
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

## write dsub file

In [ ]:
%%writefile ~/aou_dsub.bash

#!/bin/bash

# This shell function passes reasonable defaults for several dsub parameters, while
# allowing the caller to override any of them. It creates a nice folder structure within
# the workspace bucket for dsub log files.

# --[ Parameters ]--
# any valid dsub parameter flag

#--[ Returns ]--
# the job id of the job created by dsub

#--[ Details ]--
# The first five parameters below should always be those values when running on AoU RWB.

# Feel free to change the values for --user, --regions, --logging, and --image if you like.

# Note that we insert some job data into the logging path.
# https://github.com/DataBiosphere/dsub/blob/main/docs/logging.md#inserting-job-data

function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  #local AOU_NETWORK=network
  #local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-batch \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'ubuntu:latest' \
      --network "global/networks/network" \
      --subnetwork "regions/us-central1/subnetworks/subnetwork" \
      --service-account "$(gcloud config get-value account)" \
      --use-private-address \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

# subset plink files to variants in scores and individuals in sample

## copy plink files to workspace bucket

In [ ]:
!gsutil -u $GOOGLE_PROJECT -m cp gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/* ${WORKSPACE_BUCKET}/CKD/input/acaf/

In [ ]:
!gsutil -u $GOOGLE_PROJECT ls -lh gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/*

## run command

In [ ]:
import pandas as pd
import os
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'plink_subset-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data = {
    '--env CHROM': [x for x in range(1, 23)] + ["X"],
    '--output-recursive OUT_DIR': [f"{bucket}/CKD/input/acaf_pgen_subset/" for _ in range(23)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/plink_subset" \
  --disk-size 180 \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --pfile $BUCKET/CKD/input/acaf/acaf_threshold.chr$CHROM \
                      --extract range $BUCKET/CKD/input/ckd.all_weights.var_list.txt \
                      --set-all-var-ids @:#:\$r:$a \
                      --new-id-max-allele-len 1000 \
                      --make-pgen \
                      --out $OUT_DIR/acaf_threshold.ckd_pgs_weights_vars.chr$CHROM'

print("\n".join(job_output))
plink_job_id = job_output[1].split(" ")[-1]
%env JOB_ID={plink_job_id}

## check worker status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME} \
    --status '*'

## check output directory/file size size

In [ ]:
!gsutil ls -lh ${WORKSPACE_BUCKET}/CKD/input/acaf_pgen_subset/

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/CKD/input/acaf_pgen_subset/acaf_threshold.ckd_pgs_weights_vars.chr1.psam .
!wc -l acaf_threshold.ckd_pgs_weights_vars.chr1.psam

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/CKD/input/acaf_pgen_subset/acaf_threshold.ckd_pgs_weights_vars.chr1.pvar .
!wc -l acaf_threshold.ckd_pgs_weights_vars.chr1.pvar

## remove original plink files from workspace bucket

In [ ]:
!gsutil rm ${WORKSPACE_BUCKET}/CKD/input/acaf/*

# create sample sheet

## create file path list
- replace "xxxx" with rest of workspace bucket path

In [ ]:
filepath_list = []
for chr in list(range(1, 23)):
    filepath = '/mnt/data/mount/gs/fc-secure-xxxx/CKD/input/acaf_pgen_subset/acaf_threshold.ckd_pgs_weights_vars.chr' + str(chr)
    filepath_list.append(filepath)
print(len(filepath_list))
filepath_list

## create dataframe

In [ ]:
import pandas as pd
samplesheet = pd.DataFrame(data = {'sampleset' : 'AOU',
                                   'path_prefix' : filepath_list,
                                   'chrom' : list(range(1, 23)),
                                   'format' : 'pfile'})
samplesheet

## export dataframe

In [ ]:
samplesheet.to_csv('AOU.CKD.PGSC_CALC.samplesheet.csv', index = None)

## copy to workspace bucket

In [ ]:
!gsutil cp AOU.CKD.PGSC_CALC.samplesheet.csv ${WORKSPACE_BUCKET}/CKD/input/

# test docker container

## run command

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'test-docker'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data = {
    '--output-recursive OUT_DIR': [f"{bucket}/CKD/output/"]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/pgsc_calc_general:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/ckd/" \
  --boot-disk-size 20 \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'nextflow run pgscatalog/pgsc_calc -profile conda'

job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## check log files

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/dsub_logs/ckd/${JOB_ID}* .

In [ ]:
!cat ${JOB_ID}.1-stdout.log

In [ ]:
!cat ${JOB_ID}.1-stderr.log

# run PGSC-calc

## run command

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'pgsc_calc'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data = {
    '--output-recursive OUT_DIR': [f"{bucket}/CKD/output/"]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/pgsc_calc_general:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/ckd/" \
  --disk-size 512 \
  --boot-disk-size 20 \
  --min-ram 208 \
  --min-cores 32 \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'nextflow run pgscatalog/pgsc_calc -profile conda \
                  --input $BUCKET/CKD/input/AOU.CKD.PGSC_CALC.samplesheet.csv \
                  --scorefile "$BUCKET/CKD/input/pgs_weights/*" \
                  --target_build GRCh38 \
                  --outdir $OUT_DIR \
                  --max_cpus 32 \
                  --max_memory 208.GB \
                  --min_overlap 0.0 \
                  --max_time 240.h \
                  --run_ancestry $BUCKET/CKD/input/pgsc_HGDP+1kGP_v1.tar.zst \
                  --keep_multiallelic True \
                  --hwe_ref 0 \
                  --pca_maf_target 0.05'

job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check worker status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "pgsc-calc--kathleen-cardone--250613-132330-00" \
    --users "kathleen-cardone" \
    --status '*'

## check logs

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/dsub_logs/ckd/${JOB_ID}* .

In [ ]:
!cat ${JOB_ID}.1-stderr.log

In [ ]:
!cat ${JOB_ID}.1-stdout.log

# run PGSC-CALC on PRScs iterations

## make sure all input files are there

In [ ]:
!gsutil ls gs://fc-secure-b117990b-e93e-442c-a994-81f2ca68d4fb/CKD/input/PRScs_iterations_weights/ | wc -l

In [ ]:
%%bash
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/EAS.Phe_585.3/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/EAS.eGFR/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/EAS.eGFR.flip/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/EUR.Phe_585.3/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/EUR.eGFR/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/EUR.eGFR.flip/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/AFR.Phe_585.3/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/AFR.eGFR/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/AFR.eGFR.flip/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/AMR.eGFR/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/AMR.eGFR.flip/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/META.eGFR/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/META.eGFR.2/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/META.eGFR.flip/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/META.eGFR.flip.2/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/META.Phe_585.3/ | wc -l
gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/META.Phe_585.3.2/ | wc -l

In [ ]:
!gsutil ls ${WORKSPACE_BUCKET}/CKD/input/PRScs_iterations_weights/

## run command

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'pgsc_calc'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data = {
    '--env SCORE': ['AFR.eGFR', 'AFR.eGFR.flip', 'AFR.Phe_585.3', 'AMR.eGFR', 'AMR.eGFR.flip', 'EAS.eGFR', 'EAS.eGFR.flip', 'EAS.Phe_585.3', 'EUR.eGFR', 'EUR.eGFR.flip', 'EUR.Phe_585.3', 'META.eGFR', 'META.eGFR.flip', 'META.Phe_585.3', 'META.eGFR.2', 'META.eGFR.flip.2', 'META.Phe_585.3.2'],
    '--output-recursive OUT_DIR': [f"{bucket}/CKD/output/PRScs_iterations/{dir}/" for dir in ['AFR.eGFR', 'AFR.eGFR.flip', 'AFR.Phe_585.3', 'AMR.eGFR', 'AMR.eGFR.flip', 'EAS.eGFR', 'EAS.eGFR.flip', 'EAS.Phe_585.3', 'EUR.eGFR', 'EUR.eGFR.flip', 'EUR.Phe_585.3', 'META.eGFR', 'META.eGFR.flip', 'META.Phe_585.3', 'META.eGFR.2', 'META.eGFR.flip.2', 'META.Phe_585.3.2']]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/pgsc_calc_general:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/ckd/" \
  --disk-size 512 \
  --boot-disk-size 20 \
  --min-ram 208 \
  --min-cores 32 \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'nextflow run pgscatalog/pgsc_calc -profile conda \
                  --input $BUCKET/CKD/input/AOU.CKD.PGSC_CALC.samplesheet.csv \
                  --scorefile "$BUCKET/CKD/input/PRScs_iterations_weights/${SCORE}/*" \
                  --target_build GRCh38 \
                  --outdir $OUT_DIR \
                  --max_cpus 32 \
                  --max_memory 208.GB \
                  --min_overlap 0.0 \
                  --max_time 240.h \
                  --run_ancestry $BUCKET/CKD/input/pgsc_HGDP+1kGP_v1.tar.zst \
                  --keep_multiallelic True \
                  --hwe_ref 0 \
                  --pca_maf_target 0.05'

job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check worker status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## check logs

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/dsub_logs/ckd/${JOB_ID}* .

In [ ]:
!cat ${JOB_ID}.2-stderr.log

In [ ]:
!cat ${JOB_ID}.2-stdout.log

# redo meta iterations

## run command

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.', '-')
%env USER_NAME={USER_NAME}
job_output = []


JOB_NAME=f'pgsc_calc'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data = {
    '--env SCORE': ['META.Phe_585.3', 'META.eGFR.flip', 'META.eGFR.flip.2'],
    '--output-recursive OUT_DIR': [f"{bucket}/CKD/output/PRScs_iterations/{dir}/" for dir in ['META.Phe_585.3', 'META.eGFR.flip', 'META.eGFR.flip.2']]
})

PARAMETER_FILENAME = f'params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep = '\t', index = False)

job_output = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-batch \
  --image "gcr.io/ritchie-aou-psom-9015/pgsc_calc_general:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/ckd/" \
  --disk-size 512 \
  --boot-disk-size 20 \
  --min-ram 260 \
  --min-cores 32 \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'nextflow run pgscatalog/pgsc_calc -profile conda \
                  --input $BUCKET/CKD/input/AOU.CKD.PGSC_CALC.samplesheet.csv \
                  --scorefile "$BUCKET/CKD/input/PRScs_iterations_weights/${SCORE}/*" \
                  --target_build GRCh38 \
                  --outdir $OUT_DIR \
                  --max_cpus 32 \
                  --max_memory 260.GB \
                  --min_overlap 0.0 \
                  --max_time 240.h \
                  --run_ancestry $BUCKET/CKD/input/pgsc_HGDP+1kGP_v1.tar.zst \
                  --keep_multiallelic True \
                  --hwe_ref 0 \
                  --pca_maf_target 0.05'

job_id = job_output[1].split(" ")[-1]
%env JOB_ID={job_id}

## check worker status

In [ ]:
!dstat \
    --provider google-batch \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## check logs

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/dsub_logs/ckd/${JOB_ID}.2* .

In [ ]:
!cat ${JOB_ID}.2-stderr.log

In [ ]:
!cat ${JOB_ID}.2-stdout.log